# Populating Neo4j Graph Database

In [2]:
import os
import json
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

NEO4J_URL = os.getenv("NEO4J_CONNECTION_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
from graph_vector_store import setup_graph

setup_graph()

#### Creating _text_embedding_ property in all entities and populate them with Embeddings Vector

We'll use those later to execute similarity search

In [4]:
database_vector = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="database_embeddings",
    node_label="Database",
    text_node_properties=["database_description"],
    embedding_node_property="text_embeddings"
)

table_vector = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="table_embeddings",
    node_label="Table",
    text_node_properties=["table_logic_name", "table_description"],
    embedding_node_property="text_embeddings"
)

column_vector = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="column_embeddings",
    node_label="Column",
    text_node_properties=["column_name","column_logic_name", "column_description"],
    embedding_node_property="text_embeddings"
)

In [5]:
table_vector.similarity_search_with_relevance_scores("Quais os 5 produtos mais vendidos em 2018?",k=2)

[(Document(page_content='\ntable_logic_name: Tabela dimensão de produtos\ntable_description: A tabela `dim_products` representa a dimensão produtos no e-commerce Olist. Nela são catalogados todos os produtos da plataforma do e-commerce assim como informações sobre eles como a categoria dos produtos, o peso, largura, comprimento, altura, comprimento da descrição do produto e quantidade de fotos disponíveis.', metadata={'database_name': 'olist', 'table_name': 'dim_products', 'table_full_name': 'olist.dim_products', 'table_primary_key': ['product_id'], 'table_columns': ['product_id', 'product_category_name', 'product_name_length', 'product_description_length', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']}),
  0.8799126744270325),
 (Document(page_content='\ntable_logic_name: Tabela dimensão de vendedores\ntable_description: A tabela `dim_sellers` contém todos os vendedores da plataforma. A tabela contem identificação do vendedor, s

In [31]:
def fetch_relevant_tables_by_column(column_vector: Neo4jVector, prompt: str, top_k: int = 5) -> list:

    similarity_search_result = column_vector.similarity_search_with_relevance_scores(query=prompt, k=top_k)

    dict_list = [
            {
                'located_at': t[0].metadata['located_at'],
                'score': t[1]
            }
        for t in similarity_search_result
    ]

    flattened_list = [(item, entry['score']) for entry in dict_list for item in entry['located_at']]

    flattened_list.sort(key=lambda x: x[1], reverse=True)

    seen = set()
    relevant_tables = []
    for item, score in flattened_list:
        if item not in seen:
            seen.add(item)
            relevant_tables.append(item)

    return relevant_tables

fetch_relevant_tables_by_column(column_vector=column_vector, prompt='Quais os 5 produtos mais vendidos em 2018?')

['dim_order_items', 'dim_products', 'fact_orders']

In [75]:
def fetch_relevant_tables(table_vector: Neo4jVector, prompt: str, top_k: int = 5) -> list:
    
    query_result = table_vector.similarity_search_with_relevance_scores(query=prompt, k=top_k)

    dict_list = [
            {
                'table_name': t[0].metadata['table_full_name'].split('.')[1],
                'score': t[1]
            }
        for t in query_result
    ]
    relevant_tables = [x['table_name'] for x in dict_list]

    return relevant_tables


fetch_relevant_tables(table_vector=table_vector, prompt='Quais os 5 produtos mais vendidos em 2018?')

['dim_products',
 'dim_sellers',
 'dim_customers',
 'fact_orders',
 'dim_order_items']

#### Fetching Columns related to relevant Tables

In [32]:
relevant_tables = fetch_relevant_tables_by_column(prompt="Quais os 5 produtos mais vendidos em 2018?", column_vector=column_vector)

def fetch_table_columns(tables: list):

    query_string = """ 
        MATCH (tb:Table)-[:HAS_COLUMN]->(col:Column)
        WHERE tb.table_name in $relevant_tables
        WITH tb, collect({
            column_name: col.column_name,
            data_type: col.data_type,
            column_description: col.column_description
        }) AS columns
        RETURN {
            database_name: tb.database_name,
            table_name: tb.table_name,
            table_description: tb.table_description,
            table_columns: columns
        } AS table
    """

    params = {'relevant_tables': tables}

    result = graph.query(query=query_string, params=params)

    result = [x['table'] for x in result]

    return result


In [ ]:
"""
MATCH (tb1:Table)-[:HAS_COLUMN]->(col:Column)<-[:HAS_COLUMN]-(tb2:Table)
WHERE tb1.table_name in ['dim_order_items', 'dim_products', 'fact_orders']
RETURN (tb1), (col), (tb2)
"""

In [50]:
def get_sql_context(user_prompt: str) -> str:
    relevant_tables = fetch_relevant_tables_by_column(prompt=user_prompt, column_vector=column_vector)

    context = fetch_table_columns(tables=relevant_tables)

    for table in context:

        context_template = ""
        
        columns_text = ""
        for column in table['table_columns']:
            columns_text += ' '.join(['nome:', column['column_name'],'tipo:', column['data_type'], 'descrição:', column['column_description'] ])
            columns_text += '\n    '

        context_template += f"""

Database: {table['database_name']}
Tabela: {table['table_name']}
Descrição: {table['table_description']}
Colunas: 
    {columns_text}
"""

    return context_template 

context=get_sql_context(user_prompt="Quais os 5 produtos mais vendidos em 2018?")



Database: olist
Tabela: dim_products
Descrição: A tabela `dim_products` representa a dimensão produtos no e-commerce Olist. Nela são catalogados todos os produtos da plataforma do e-commerce assim como informações sobre eles como a categoria dos produtos, o peso, largura, comprimento, altura, comprimento da descrição do produto e quantidade de fotos disponíveis.
Colunas: 
    nome: product_width_cm tipo: float descrição: A coluna `product_width_cm` apresenta a largura do produto em centímetros.
    nome: product_height_cm tipo: float descrição: A coluna `product_height_cm` apresenta a altura do produto em centímetros.
    nome: product_length_cm tipo: float descrição: A coluna `product_length_cm` apresenta o comprimento do produto em centímetros.
    nome: product_weight_g tipo: float descrição: A coluna `product_weight` apresenta o peso em gramas do produto.
    nome: product_photos_qty tipo: integer descrição: A coluna `product_photos_qty` contém a quantidade de fotos que existe no

#### Creating Langchain Custom Retriever

In [ ]:
from langchain.callbacks import BaseCallback

class CustomFormatCallback(BaseCallback):
    def on_retrieve(self, result, **kwargs):
        # Customize the format of the retrieval result here
        customized_result = [self.customize(entry) for entry in result]
        return customized_result

    def customize(self, entry):
        # Define your custom format logic here
        return {
            'id': entry['id'],
            'custom_field': entry['some_other_field'],
            # add other custom fields as needed
        }

# Use the callback when creating the retriever


In [74]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.callbacks import BaseCallback
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

from langchain.globals import set_verbose

set_verbose(True)

model = ChatOpenAI(model="gpt-3.5-turbo")

class RetrievalFormatCallback(BaseCallback):

    def on_retrieve(self, result, **kwargs):

        list_of_tables_list =  [entry.metadata['located_at'] for entry in result]

        relevant_tables_duplicated = [item for sublist in list_of_tables_list for item in sublist]

        relevant_tables = list(set(relevant_tables_duplicated))

        query_string = """ 
        MATCH (tb:Table)-[:HAS_COLUMN]->(col:Column)
        WHERE tb.table_name in $relevant_tables
        WITH tb, collect({
            column_name: col.column_name,
            data_type: col.data_type,
            column_description: col.column_description
        }) AS columns
        RETURN {
            database_name: tb.database_name,
            table_name: tb.table_name,
            table_description: tb.table_description,
            table_columns: columns
        } AS table
        """

        params = {'relevant_tables': relevant_tables}

        NEO4J_URL = os.getenv("NEO4J_CONNECTION_URL")
        NEO4J_USERNAME = os.getenv("NEO4J_USER")
        NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

        graph = Neo4jGraph(
            url=NEO4J_URL,
            username=NEO4J_USERNAME,
            password=NEO4J_PASSWORD
        )

        result = graph.query(query=query_string, params=params)

        result = [x['table'] for x in result]

        for table in result:

            context_template = ""
            
            columns_text = ""
            for column in table['table_columns']:
                columns_text += ' '.join(['nome:', column['column_name'],'tipo:', column['data_type'], 'descrição:', column['column_description'] ])
                columns_text += '\n    '

            context_template += f"""
Database: {table['database_name']}
Tabela: {table['table_name']}
Descrição: {table['table_description']}
Colunas: 
    {columns_text}
"""

        return context_template 
    

retriever = table_vector.as_retriever(callbacks=[RetrievalFormatCallback()])

class SqlResponse(BaseModel):
    input: str = Field(description="O prompt original do usuário.")
    output: str = Field(description="O código SQL que responde a pergunta do usuário.")

parser = PydanticOutputParser(pydantic_object=SqlResponse)

system_message = """
Você é um especialista em SQL responsável por gerar uma consulta SQL
compatível com Trino e que responda a pergunta do usuário.
Sua consulta deverá ser formatada, tabulada e utilizar aliases para
evitar ambiguidades.
Sua resposta devera ser apenas código SQL, não complemente sua resposta.

Utilize o contexto fornecido para construir a consulta SQL:
{context}

Responda a pergunta do usuário. Armazena sua resposta em um objeto JSON.
{format_instructions}
"""

user_message = """
{question}
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("user", user_message)
    ]
).partial(format_instructions=parser.get_format_instructions())

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt_template | model | parser

ImportError: cannot import name 'BaseCallback' from 'langchain.callbacks' (c:\Users\Gabriel Marques\github\graphrag\.venv\Lib\site-packages\langchain\callbacks\__init__.py)

In [72]:
response = chain.invoke("Quais os 5 produtos mais vendidos em 2018?",  config={'callbacks': [ConsoleCallbackHandler()]})
print(response.output)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Quais os 5 produtos mais vendidos em 2018?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "Quais os 5 produtos mais vendidos em 2018?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Quais os 5 produtos mais vendidos em 2018?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Quais os 5 produtos mais vendidos em 2018?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question>] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTempla

In [8]:

graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database='neo4j'
)

In [9]:
schema_description = graph.query(
    query="CALL apoc.ml.schema({apiKey: $openai_api_key, model: 'gpt-4o'}) yield value RETURN *",
    params={"openai_api_key": OPENAI_API_KEY}
    )

In [10]:
print(schema_description[0]["value"])

This database schema models information about databases and tables. 
1. **Database Node:** Represents a database with descriptions and text embeddings, like a library.
2. **Table Node:** Represents a table within a database, including columns, primary keys, and descriptions.
3. **CONTAINS Relationship:** Links a database to the tables it holds, similar to library sections.
4. **HAS_COLUMN Relationship:** Connects tables to their columns, analogous to chapters within a book.
5. **Patterns:** Visualize databases containing tables, and tables having columns, like a nested file system.


## Creating Agents

In [395]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import List
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatOpenAI(model='gpt-4o')


In [14]:
print(graph.get_schema)

Node properties:
Database {database_description: STRING, database_name: STRING, database_tables: LIST, text_embeddings: LIST}
Table {table_logic_name: STRING, text_embeddings: LIST, table_name: STRING, table_columns: LIST, table_primary_key: LIST, database_name: STRING, table_full_name: STRING, table_description: STRING}
Column {text_embeddings: LIST, data_type: STRING, is_primary_key_at: LIST, is_foreign_key_at: LIST, column_name: STRING, column_description: STRING, located_at: LIST}
Relationship properties:

The relationships:
(:Database)-[:CONTAINS]->(:Table)
(:Table)-[:HAS_COLUMN]->(:Column)
(:Column)-[:IS_FOREIGN_KEY]->(:Table)
(:Column)-[:IS_PRIMARY_KEY]->(:Table)


#### Query Expansion Agent

In [51]:
class QueryExpansionResponse(BaseModel):
    input: str = Field(description="The original user prompt.")
    output: List[str] = Field(description="A list of sentences.")

parser = PydanticOutputParser(pydantic_object=QueryExpansionResponse)

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
Você é um especialista em parafrasear frases.
O usuário irá te enviar uma pergunta ou solicitação
e você deverá parafraseá-la.
Mantenha o sentido original da frase.
As frases devem ser em português brasileiro.
Gere pelo menos 5 frases diferentes.
Sua resposta deverá ser apenas a lista de frase separadas por vírgulas.
Não complemente ou formate sua resposta com caracteres especiais ou de escape.
"Answer the user query. Wrap the output in `json` tags\n{format_instructions}"
     """),
    ("user", "{input}")
]).partial(format_instructions=parser.get_format_instructions())

query_expansion_chain = prompt | llm | parser


NameError: name 'BaseModel' is not defined

In [401]:
paraphrased_prompt = query_expansion_chain.invoke({"input": "Quais os 5 produtos mais vendidos em 2018?"})
paraphrased_prompt.output

['Quais foram os 5 produtos mais populares em vendas em 2018?',
 'Quais são os cinco itens com maior número de vendas em 2018?',
 'Quais os cinco produtos com mais comercialização em 2018?',
 'Quais foram os produtos mais vendidos no ano de 2018?',
 'Quais são os top 5 produtos mais vendidos em 2018?']

In [397]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
Você é um especialista em análise de negócios e investigação.
O usuário irá te enviar uma pergunta ou solicitação.
Você deve criar pelo menos outras 5 perguntas que colaborem
para o usuário tenha sua pergunta respondida.

"Answer the user query. Wrap the output in `json` tags\n{format_instructions}"
     """),
    ("user", "{input}")
]).partial(format_instructions=parser.get_format_instructions())

query_expansion_chain = prompt | llm | parser

In [398]:
paraphrased_prompt = query_expansion_chain.invoke({"input": "Quais os 5 produtos mais vendidos em 2018?"})
paraphrased_prompt.output

['Qual é o setor ou indústria de interesse?',
 'A que região ou país você está se referindo?',
 'Você está interessado em produtos físicos, digitais ou ambos?',
 'Você tem uma fonte de dados específica ou um site de referência em mente?',
 'Você está se referindo a unidades vendidas ou ao valor de vendas?']

#### SQL Agent

In [409]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub




In [412]:
sql_prompt_template = ChatPromptTemplate.from_messages([
    ("system", 
     """
Você é um especialista em SQL responsável por gerar uma consulta SQL
compatível com Trino e que responda a pergunta do usuário.
Sua consulta deverá ser formatada, tabulada e utilizar aliases para
evitar ambiguidades. Caso precise gerar uma ou mais subqueries,
prefira utilizar CTEs (Common Table Expressions).
Sua resposta devera ser apenas código SQL, não complemente sua resposta.

Utilize o contexto fornecido para construir a consulta SQL:
{context}

Answer the user query. Wrap the output in `json` tags
{format_instructions} """),
    ("user", "{input}")
]).partial(format_instructions=parser.get_format_instructions())

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

sql_agent = create_retrieval_chain(retriever=table_vector.as_retriever(), combine_docs_chain=combine_docs_chain)

sql_agent.invoke(
    {
        "input": "Quais os 5 produtos mais vendidos em 2018?",
        "context": 
    }
)


{'input': 'Quais os 5 produtos mais vendidos em 2018?',
 'context': [Document(page_content='\ntable_name: dim_products\ntable_logic_name: Tabela dimensão de produtos\ntable_description: A tabela `dim_products` representa a dimensão produtos no e-commerce Olist. Nela são catalogados todos os produtos da plataforma do e-commerce assim como informações sobre eles como a categoria dos produtos, o peso, largura, comprimento, altura, comprimento da descrição do produto e quantidade de fotos disponíveis.', metadata={'database_name': 'olist', 'table_full_name': 'olist.dim_products', 'table_primary_key': ['product_id'], 'table_columns': ['product_id', 'product_category_name', 'product_name_length', 'product_description_length', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']}),
  Document(page_content='\ntable_name: dim_sellers\ntable_logic_name: Tabela dimensão de vendedores\ntable_description: A tabela `dim_sellers` contém todos os vende